<a href="https://colab.research.google.com/github/kridtapon/MACD-R-Swing-Strategy/blob/main/MACD_R_Swing_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.3 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    df['MACD'] = df['Close'].ewm(span=short_window, min_periods=1).mean() - df['Close'].ewm(span=long_window, min_periods=1).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, min_periods=1).mean()
    return df

# Function to calculate Williams %R
def calculate_williams_r(df, period=14):
    highest_high = df['High'].rolling(window=period).max()
    lowest_low = df['Low'].rolling(window=period).min()
    df['Williams_%R'] = -100 * (highest_high - df['Close']) / (highest_high - lowest_low)
    return df

# Define the stock symbol and time period
symbol = 'META'  # Example stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate MACD and Williams %R
df = calculate_macd(df)
df = calculate_williams_r(df, period=14)

# Define Entry and Exit signals based on MACD + Williams %R Strategy
df['Entry'] = (df['MACD'] > df['MACD_Signal']) & (df['Williams_%R'] < -20)
df['Exit'] = (df['MACD'] < df['MACD_Signal']) & (df['Williams_%R'] < -80)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                            95529.221768
Total Return [%]                        -4.470778
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                       7397.579013
Max Drawdown [%]                        74.691187
Max Drawdown Duration                      1093.0
Total Trades                                   38
Total Closed Trades                            38
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            34.210526
Best Trade [%]                          52.710279
Worst Trade [%]                        -27.105312
Avg Winning Trade [%]                   15.697971
Avg Losing Trade [%]                    -6.731214
